# Product Recommendation with Feathr

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

### Model Problem Statement
The e-commerce website has collected past user ratings for various products. The website also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

### Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by `user_id` and `product_id`. With that, we can think about three types of features:
1. **User features** that are different for different users but are the same for different products. For example, user age is different for different users but it's product-agnostic.
2. **Product features** that are different for different products but are the same for all the users.
3. **User-to-product** features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

In this example, we will focus on the first two types of features. After we train a model based on those features, we predict the product ratings that users will give for the products.

The feature creation flow is as below:
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

## 2. Config Feathr Client

In [1]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
from pyspark.sql import DataFrame
# from interpret.provider import InlineProvider
# from interpret import set_visualize_provider

# set_visualize_provider(InlineProvider())

import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df

print(f"Feathr version: {feathr.__version__}")

Feathr version: 1.0.0


In [2]:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"
os.environ['REDIS_PASSWORD'] = "foobared"
PROJECT_NAME = "product_recommendation_from_s3"

### Initialize Feathr Client

In [3]:
from pathlib import Path
feathr_workspace_folder = Path(f"./{PROJECT_NAME}_feathr_config.yaml")
client = FeathrClient(str(feathr_workspace_folder))

2024-09-06 21:00:49.047 | INFO     | feathr.utils._env_config_reader:get:62 - Config secrets__azure_key_vault__name is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 21:00:49.050 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__s3__s3_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 21:00:49.051 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__adls__adls_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 21:00:49.053 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__wasb__wasb_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 21:00:49

## 3. Prepare Datasets

In [4]:
# Download datasets
WORKING_DIR = PROJECT_NAME
user_observation_file_path = "s3a://product-recommendation/user_observation.csv"
user_profile_file_path = "s3a://product-recommendation/user_profile.csv"
user_purchase_history_file_path = "s3a://product-recommendation/user_purchase_history.csv"
product_detail_file_path = "s3a://product-recommendation/product_detail.csv"

user_observation_source_path = user_observation_file_path
user_profile_source_path = user_profile_file_path
user_purchase_history_source_path = user_purchase_history_file_path
product_detail_source_path = product_detail_file_path

In [5]:
import pandas as pd
import numpy as np
import random

## 4. Define Sharable Features using Feathr API

### What's a feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive.

In Feathr, a feature is defined by the following characteristics:
* The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
* The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
* The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.
* The timestamp: this indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

You can feel that this is defined from a feature consumer (a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define how to produce the feature, we need to specify:
* Feature source: what source data that this feature is based on
* Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html).)

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields.

### Define window aggregation features

[Window aggregation](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) helps us to create more powerful features by compressing large amount of information. For example, we can compute *average purchase amount over the last 90 days* from the purchase history to capture user's recent consumption trend.

To create window aggregation features, we define `WindowAggTransformation` with following arguments:
1. `agg_expr`: the field/column you want to aggregate. It can be an ANSI SQL expression, e.g. `cast_float(purchase_amount)` to cast `str` type values to `float`.
2. `agg_func`: the aggregation function, e.g. `AVG`. See below table for the full list of supported functions.
3. `window`: the aggregation window size, e.g. `90d` to aggregate over the 90 days.

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
| `SUM`, `COUNT`, `MAX`, `MIN`, `AVG` | Numeric | Applies the the numerical operation on the numeric inputs. |
| `MAX_POOLING`, `MIN_POOLING`, `AVG_POOLING`	| Numeric Vector | Applies the max/min/avg operation on a per entry basis for a given a collection of numbers. |
| `LATEST` | Any | Returns the latest not-null values from within the defined time window. |

After you have defined features and sources, bring them together to build an anchor:

> Note that if the features comes directly from the observation data, the `source` argument should be `INPUT_CONTEXT` to indicate the source of the anchor is the observation data.

## Get features from Register Server

In [6]:
client.list_registered_features(project_name=PROJECT_NAME)

[{'name': 'feature_user_age',
  'id': '4ab9154c-e5ec-4897-8744-6e5731e40f96',
  'qualifiedName': 'product_recommendation_from_s3__anchored_features__feature_user_age'},
 {'name': 'feature_user_gift_card_balance',
  'id': '6b41d544-d983-46f6-ad1b-8b04afb75467',
  'qualifiedName': 'product_recommendation_from_s3__anchored_features__feature_user_gift_card_balance'},
 {'name': 'feature_product_quantity',
  'id': 'a00b54ca-cfcb-4e65-b36d-8890cc5221c6',
  'qualifiedName': 'product_recommendation_from_s3__product_anchored_features__feature_product_quantity'},
 {'name': 'feature_user_purchasing_power',
  'id': '3d611a32-883e-4be1-ae58-45dfc4cdbf7e',
  'qualifiedName': 'product_recommendation_from_s3__feature_user_purchasing_power'},
 {'name': 'feature_user_avg_purchase_for_90days',
  'id': 'fff49cb3-2e0c-45ae-bd60-093d868e2b78',
  'qualifiedName': 'product_recommendation_from_s3__aggregationFeatures__feature_user_avg_purchase_for_90days'},
 {'name': 'feature_user_tax_rate',
  'id': '2d344b29-7

In [7]:
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, 
                                                 return_keys=True, 
                                                 )

## we can list all features

In [8]:
[feat.name for feat in list(feature_dict[0].values())]

['feature_user_avg_purchase_for_90days',
 'feature_user_age',
 'feature_user_tax_rate',
 'feature_user_gift_card_balance',
 'feature_user_has_valid_credit_card',
 'feature_product_quantity',
 'feature_product_price',
 'feature_user_purchasing_power']

## we can list all type_key

In [9]:
[type_key.key_column for type_keys in list(feature_dict[1].values()) for type_key in type_keys]

['user_id',
 'user_id',
 'user_id',
 'user_id',
 'user_id',
 'product_id',
 'product_id',
 'user_id']

## 5. Create Training Data using Point-in-Time Correct Feature join

To create a training dataset using Feathr, we need to provide a **feature join settings** to specify what features and how these features should be joined to the observation data.

Also note that since a `FeatureQuery` accepts features of the same join key, we define two query objects, one for `user_id` key and the other one for `product_id` and pass them together to compute offline features.

To learn more on this topic, please refer to [Point-in-time Correctness document](https://feathr-ai.github.io/feathr/concepts/point-in-time-join.html).

In [10]:
key_user_id = feature_dict[1]["feature_user_avg_purchase_for_90days"][0]
key_product_id = feature_dict[1]["feature_product_quantity"][0]

In [11]:
user_feature_query = FeatureQuery(
    feature_list=[feat.name for feat in list(feature_dict[0].values()) if "user" in feat.name], #[feat.name for feat in features + agg_features + derived_features],
    key=key_user_id, #user_id
)

product_feature_query = FeatureQuery(
    feature_list= [feat.name for feat in list(feature_dict[0].values()) if "product" in feat.name] ,#[feat.name for feat in product_features],
    key=key_product_id, #product_id
)


In [18]:
os.environ['S3_ENDPOINT'] = "127.0.0.1:9000"
os.environ['S3_ACCESS_KEY'] = "zDaLfup26Qf2psWSVksV"
os.environ['S3_SECRET_KEY'] = "0uP2ohdTWeLXzV5UAlF8z5JVn17aEbr6OmOa9dkv"

In [25]:
settings = ObservationSettings(
    observation_path=user_observation_source_path,
    event_timestamp_column="event_timestamp",
    timestamp_format="yyyy-MM-dd",
)
client.get_offline_features(
    observation_settings=settings,
    feature_query=[user_feature_query, product_feature_query],
    output_path="s3a://product-recommendation/trainingdata",
    execution_configurations={"spark.feathr.outputFormat": "parquet",
                              "spark.hadoop.fs.s3a.endpoint": "http://127.0.0.1:9000",
                              "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
                              "spark.hadoop.fs.s3a.path.style.access": "true",
                              'spark.hadoop.fs.s3a.aws.credentials.provider':'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider',
                              "spark.jars.plus.packages": 
                                                       "io.minio:spark-select_2.11:2.1,org.apache.hadoop:hadoop-aws:3.3.2"
                             
                             }
)
client.wait_job_to_finish(timeout_sec=100000)

2024-09-06 22:38:20.130 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Local Spark Mode only support basic params right now and should be used only for testing purpose.
2024-09-06 22:38:20.132 | INFO     | feathr.spark_provider._localspark_submission:_get_debug_file_name:296 - Spark log path is debug/product_recommendation_from_s3_feathr_feature_join_job20240906223820
2024-09-06 22:38:20.137 | INFO     | feathr.spark_provider._localspark_submission:_init_args:271 - Spark job: product_recommendation_from_s3_feathr_feature_join_job is running on local spark with master: local[*].
2024-09-06 22:38:20.213 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:151 - Detail job stdout and stderr are in debug/product_recommendation_from_s3_feathr_feature_join_job20240906223820/log.
2024-09-06 22:38:20.219 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:161 - Local Spark job submit with pid: 90577.
2024-09-06 22

x

https://repository.mulesoft.org/nexus/content/repositories/public/ added as a remote repository with the name: repo-1
https://linkedin.jfrog.io/artifactory/open-source/ added as a remote repository with the name: repo-2
Ivy Default Cache set to: /Users/cuongtranhuu/.ivy2/cache
The jars for the packages stored in: /Users/cuongtranhuu/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.logging.log4j#log4j-core added as a dependency
com.typesafe#config added as a dependency
org.apache.hadoop#hadoop-mapreduce-client-core added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.avro#avro added as a dependency
org.apache.xbean#xbean-asm6-shaded added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
com.jasonclawson#jackson-dataformat-hocon added as a dependency
com.redislabs#spark-redis_2.12 added as a dependency
org.apache.xbean#xbean-asm6-shaded added 

>>>>>>>>>x>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>x>>>>xx

2024-09-06 22:41:52.765 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:198 - Pyspark job Completed


>

2024-09-06 22:41:53.778 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:234 - Spark job with pid 90577 finished in: 213 seconds.


>

Let's use the helper function `get_result_df` to download the result and view it:

In [47]:
import pyarrow.parquet as pq
import s3fs
s3 = s3fs.S3FileSystem(
    anon=False,
    use_ssl=False,
    client_kwargs={
        "endpoint_url": "http://127.0.0.1:9000",
        "aws_access_key_id": os.getenv('S3_ACCESS_KEY'),
        "aws_secret_access_key": os.getenv('S3_SECRET_KEY'),
        "verify": True,
    }
)
s3_filepath = "s3a://product-recommendation/trainingdata"

s3_filepaths = [path for path in s3.ls(s3_filepath)
                if path.endswith('.parquet')]

res_df = pq.ParquetDataset(s3_filepaths, filesystem=s3).read_pandas().to_pandas()

In [48]:
res_df

,user_id,product_id,event_timestamp,Product_rating,feature_user_avg_purchase_for_90days,feature_product_price,feature_product_quantity,feature_user_gift_card_balance,feature_user_has_valid_credit_card,feature_user_tax_rate,feature_user_age,feature_user_purchasing_power
0,415,42,2023-10-22,2,NaN,16.452332,77.0,1188.0,True,0.02,42,1288.0
1,92,88,2022-08-13,5,NaN,187.898758,32.0,1715.0,True,0.05,54,1815.0
2,346,59,2022-07-13,3,NaN,81.810997,41.0,1786.0,True,0.01,54,1886.0
3,325,4,2023-10-18,2,NaN,52.391521,98.0,1814.0,True,0.05,31,1914.0
4,279,9,2024-03-03,1,NaN,170.658722,2.0,1095.0,True,0.03,26,1195.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,216,41,2022-12-29,2,63.040001,76.591248,18.0,1901.0,True,0.01,31,2001.0
996,415,22,2023-09-23,2,NaN,189.393814,84.0,1188.0,True,0.02,42,1288.0
997,471,18,2022-05-08,3,NaN,28.654425,73.0,1190.0,True,0.03,47,1290.0
998,212,35,2022-06-18,3,NaN,42.445522,25.0,1819.0,True,0.03,19,1919.0


### Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr. Here, we use **EBM (Explainable Boosting Machine)** regressor from [InterpretML](https://github.com/interpretml/interpret) package to visualize the modeling results.

In [36]:
from interpret import show
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Fill None values with 0
final_df = (
    res_df
    .drop(["event_timestamp"], axis=1, errors="ignore")
    .fillna(0)
)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    final_df.drop(["Product_rating"], axis=1),
    final_df["Product_rating"].astype("float64"),
    test_size=0.2,
    random_state=42,
)

ebm = ExplainableBoostingRegressor()
ebm.fit(X_train, y_train)

# show(ebm_global) # Will run on 127.0.0.1/localhost at port 7080
# Note, currently InterpretML's visualization dashboard doesn't work w/ VSCODE notebook viewer
# https://github.com/interpretml/interpret/issues/317
ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7001/140433530851632/ -->

In [37]:
# Predict and evaluate
y_pred = ebm.predict(X_test)
rmse = sqrt(mean_squared_error(y_test.values.flatten(), y_pred))

print(f"Root mean squared error: {rmse}")

Root mean squared error: 1.7812847121745756


In [38]:
X_test.columns

Index(['user_id', 'product_id', 'feature_user_avg_purchase_for_90days',
       'feature_product_price', 'feature_product_quantity',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power'],
      dtype='object')